# Using MONAI to unlock clinically valuable insights from Digital Pathology

## Part 3 - Analysis of Whole Slide Image Nuclei 

<img src="images/rapids.png" alt="RAPIDS" style="width: 200px;"/>

Now that we have learned how to extract nucleus types and locations from Whole Slide images, we can load up the inference results from the whole image and take a look at a few of the methods that are suitable for this type of data. Rather than using a standard Pandas dataframe, we are going to use the RAPIDS equivalent, cuDF (CUDA dataframe). This loads the data into GPU memory rather than using the host system's RAM. As you will see, this opens up a new realm of possibilities because of the huge speed boost this can provide.

As usual, we start by importing the libraries that we'll need. As you will notice, there are a few new names here, such as cuDF, cuGraph and cuML. These are the core of the RAPIDS tools offering GPU accelerated Dataframe functionality, GPU accelerated graph analytics and GPU accelerated Machine Learning routines

You will find documentation on all of these libraries and features here https://docs.rapids.ai/api

In [ ]:
from cuml import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
from cuml.neighbors import NearestNeighbors
import cuxfilter
from cuxfilter import DataFrame, layouts
import seaborn as sns
import pandas as pd
import cudf
from cudf import DataFrame
import cugraph
import cuml
import numpy as np

We start by loading a pickled centroids list, the same as the one we just plotted in the previous exercise, but for the whole slide. It gets loaded into a pandas dataframe and then you can see that we immediately create a cuDF version of the dataframe, which puts it on to the GPU. The cdf.head command loads the first few rows and you can see that we have the nucleus types and row and column (coordinates).

In [ ]:
# import the data we saved in our other notebook
df = pd.read_pickle('data/centroids.pkl')

#create a cuda dataframe (GPU) from the pandas dataframe (CPU)
cdf = cudf.from_pandas(df)

cdf.head

We can also display some summary statistics for the dataframe columns

In [ ]:
# get some stats on the data
cdf.describe()

To get a view of the data we can use MatplotLib to show us a scatter-plot using the x and y coordinates of the nuclei, using the type to set the color of each point in the output. As you can see, this provides a clear indication of the distribution of the various types of nuclei across the tissue.

In [ ]:
# create a matplotlib colormap to match the previous notebook
cmap = ListedColormap(["blue", "gold", "lawngreen", "red"])

fig, ax = plt.subplots(1, figsize = (8, 6))
plt.scatter(df['row'], df['col'], s = 2, c = df['type'], cmap = cmap)

plt.title('Nuclei coloured by type')
plt.show()

This is certainly an interesting visualisation in its own right, but it doesn't tell us much about the relationships between the cells at a more granular level.

So, what we are going to do now is to create a graph of all the nuclei that we detected and classified in the last notebook. 
In order to construct the graph we are going to use the Nearest Neighbor algorithm to find the 5 nearest neighbors to each nucleus.

To do this we will need to import some classes from cuNN (A CUDA library for nearest neighbor computation) that is part of the RAPIDS family. 

In [ ]:
from cuml.neighbors import NearestNeighbors

knn_cuml = NearestNeighbors()
knn_cuml.fit(cdf)

%time D_cuml, I_cuml = knn_cuml.kneighbors(cdf, 5)
I_cuml, D_cuml

What this has produced is a list of the 5 nearest neighbours for each nucleus - using the row and col values, which relate to the position of the centroid of each nucleus in pixel space. The KNN (k-nearest neighbours) algorithm is using a Euclidean distance calculation (other algorithms can be used) which tells us how close each node is to every other node. Because we chose 5 as the number of nearest neighbours, we have a row value which represents the index of each node and then five columns containing the indexes of the 5 nearest nodes, with the nearest in column 0 and the furthest in column 4. You will also notice that the index in the nearest column, 0, always matches the row index. That's because the algorithm does not exclude each node from being its own nearest neighbour. We can ignore that column.

We are looking at the indexes in the I_cuml dataframe and the physical distances, in pixels, in the D_cuml dataframe.

If you want to compare this with the sklearn CPU implementation, be aware that it can take > 30 minutes to run! It is not necessary to execute it - the code is there as a reference. 

In [ ]:
# Only run this and the following cell to compare with CPU version
from sklearn.neighbors import NearestNeighbors as skNN

df_1=df[cdf]
knn_sk = skNN(algorithm='brute',n_jobs=1)
knn_sk.fit(df_1)

# Only uncomment the lines below if you have a spare 30 mins to wait for it to complete!
#%time D_sk, I_sk = knn_sk.kneighbors(df_1, 5)
#I_sk

In order to convert the output of the KNN operation into a graph, we need to prepare the data. The data needs to be presented to the RAPIDS graph library, cugraph, as a set of edges with the source and destination node and an (optional) weight parameter.

Firstly, we combine the nearest neighbor indexes and distances into one dataframe and give them unique column names. We do this so that we can use the distance to set the weight of the connection between the nodes

In [ ]:
# give the columns names because they have to be unique in the merged dataframe
# Indexes of neighbours
I_cuml.columns=['ix1','n1','n2','n3','n4'] 
# Distance to neighbours
D_cuml.columns=['ix2','d1','d2','d3','d4'] 
# Concatenate the columsn into a single dataframe
all_cols = cudf.concat([I_cuml, D_cuml],axis=1)

# remove the index and distance from the self-referenced nearest neighbour
all_cols = all_cols[['n1','n2','n3','n4','d1','d2','d3','d4']]

all_cols 

The next step is to manipulate this data so that it is in the desired format. There should be 3 columns, named 'source', 'target' and 'distance'.

To do this, you will need to extract 4 sets of columns - one for each neighbor - and then concatenate the rows into a new dataframe.

Remember that each row index represents a node, the n1-n4 columns contain the row index of a destination node and the d1-d4 columns contain the distance between these nodes. 

In [ ]:
# Reformat the data to match the way edges are defined in cuGraph
all_cols['index1'] = all_cols.index

c1 = all_cols[['index1','n1','d1']]
c1.columns=['source','target','distance']
c2 = all_cols[['index1','n2','d2']]
c2.columns=['source','target','distance']
c3 = all_cols[['index1','n3','d3']]
c3.columns=['source','target','distance']
c4 = all_cols[['index1','n4','d4']]
c4.columns=['source','target','distance']
                 
edges = [c1,c2,c3,c4]

edge_df = cudf.concat(edges)

# remove the old dataframe from memory
del(all_cols)

edge_df = edge_df.reset_index()
edge_df = edge_df[['source','target','distance']]
edge_df

Next, we need to set a maximum distance between connected nodes, so that we exclude any connections beyond a certain threshold. This may reveal groups of cells that are locally connected but separate from other 'cliques'. We will use a distance of 40 pixels to start off with but you can experiment with this setting.
Note that the distance calculation actually uses the distance squared (to save many expensive sqrt operations).

In [ ]:
edge_df = edge_df.loc[edge_df["distance"] < 40]
edge_df

This dataframe is now ready to be used to generate the graph. For this we use the cugraph library

In [ ]:
# now we can actually create a graph!!
G = cugraph.Graph()

%time G.from_cudf_edgelist(edge_df,source='source', destination='target', edge_attr='distance', renumber=True)

Once we have the graph we can do standard graph analytical operations. Triangle Count is the number of cycles of length three. A k-core of a graph is a maximal subgraph that contains nodes of degree k or more. 

In [ ]:
#now we can compute some graph metrics
count = cugraph.triangle_count(G)
print("No of triangles = " + str(count))

coreno = cugraph.core_number(G)
print("Core Number = " + str(coreno))

**Visualising the graph**

One powerful feature enabled by the RAPIDS API is the visualisation of large networks. To show this in action we are going to create a chart that displays all the nuclei centroids along with the edges between their nearest neighbors.

To do this we need two dataframes: One containing the nodes and their coordinates and the other with the edges and their source and target nodes

In [ ]:
# we only need the index of the source and target nodes

# The indexes of the source and target nodes that form an edge
edge_df = edge_df[['source','target']]

# The x and y coordinates of each node (nucleus)
nodes_ = cdf[['row','col']]
# Vertex refers to the index of an item in the 
nodes_['vertex']=nodes_.index
nodes_.columns=['x','y','vertex']

Finally we use cuXFilter to render the whole graph!

In [ ]:
cux_df = cuxfilter.DataFrame.load_graph((nodes_, edge_df))

chart0 = cuxfilter.charts.graph(edge_color_palette=['gray', 'black'],
                                            timeout=200, 
                                            node_aggregate_fn='mean', 
                                            node_color_palette=['blue'],
                                            edge_render_type='direct',
                                            edge_transparency=0.5
                                          )
d = cux_df.dashboard([chart0], layout=cuxfilter.layouts.double_feature)

# draw the graph
chart0.view()

You should be able to use the mouse-wheel to zoom in and out of the graph plot. If you zoom in far enough you will see the individual vertices (coloured) and edges (grey lines)

**Exercises**

Can you create four plots for each of the nucleus types? To do this, you will need to adapt the code shown above and create a filter that only includes the desired class.

Create a function into which you can pass some parameters such as distance and nucleus type.


In [ ]:
def render_graph(nucleus_type=1, distance_threshold=20):
    pass

    # TODO use the code above to create a general function
    # That will allow you to change the distance threshold
    # and the nucleus type
    
render_graph()

([Solution](solutions/render_graph.py))

How do the different nucleus types differ in terms of their metrics?

Create another function that plots a chart of metrics such as mean core_number for each nucleus type

In [ ]:
def plot_core_numbers(distance_threshold=20, vertex_counts=3):
    pass

    # TODO use the code above to create a general function
    # That will allow you to change the distance threshold
    # and the nucleus type
    
plot_core_numbers()

([Solution](solutions/plot_metrics.py))

We hope you enjoyed this course and discovered a few new techniques to apply to your own imaging challenges!